In [128]:
import pandas as pd
from bcb import sgs, Expectativas
from datetime import datetime, timedelta
import jinja2

In [129]:
# função para fazer a formatação das DataFrames

def formata_df(df: pd.DataFrame, format_data_rf: str, valor: str):
    
    # Formatando as datas
    df['DataReferencia'] = pd.to_datetime(df['DataReferencia'], format=format_data_rf)

    # Colocando DataReferencia como header
    df_pivot = df.pivot_table(index='Data', columns='DataReferencia', values=valor)
    
    # Formatando data e ordem da DataReferencia
    df_pivot = df_pivot.sort_index(axis=1)
    df_pivot.columns = df_pivot.columns.strftime('%Y')
    return df_pivot

In [151]:
# EndPoints

em = Expectativas()
em.describe()
ep_top5_anual = em.get_endpoint('ExpectativasMercadoTop5Anuais')
ep_m_anual = em.get_endpoint('ExpectativasMercadoAnuais')
ep_12meses = em.get_endpoint('ExpectativasMercadoInflacao12Meses')
ep_24meses = em.get_endpoint('ExpectativasMercadoInflacao24Meses')


EntitySets:
  ExpectativasMercadoTop5Anuais
  ExpectativaMercadoMensais
  ExpectativasMercadoInflacao24Meses
  ExpectativasMercadoInflacao12Meses
  ExpectativasMercadoSelic
  ExpectativasMercadoTop5Selic
  ExpectativasMercadoTop5Mensais
  ExpectativasMercadoTrimestrais
  ExpectativasMercadoAnuais


In [131]:
# IPCA top5 anual MedioPrazo
ipca_top5_anual = ep_top5_anual.query().filter(ep_top5_anual.Indicador == 'IPCA', ep_top5_anual.tipoCalculo == 'M', ep_top5_anual.Data >= '2023-01-01').select(ep_top5_anual.Indicador, ep_top5_anual.Data, ep_top5_anual.Mediana, ep_top5_anual.DataReferencia).collect()
ipca_top5_anual_pivot = formata_df(ipca_top5_anual, '%Y', 'Mediana')

In [146]:
# IPCA Media anual
ipca_media_anual = ep_m_anual.query().filter(ep_m_anual.Indicador == 'IPCA', ep_m_anual.Data >= '2024-02-29', ep_m_anual.baseCalculo == '0').select(ep_m_anual.Indicador, ep_m_anual.Data, ep_m_anual.DataReferencia, ep_m_anual.Media, ep_m_anual.baseCalculo).collect()
ipca_media_anual_pivot = formata_df(ipca_media_anual, '%Y', 'Media')
ipca_media_anual_pivot

# IPCA Mediana anual
ipca_mediana_anual = ep_m_anual.query().filter(ep_m_anual.Indicador == 'IPCA', ep_m_anual.Data >= '2024-02-29', ep_m_anual.baseCalculo == '0').select(ep_m_anual.Indicador, ep_m_anual.Data, ep_m_anual.DataReferencia, ep_m_anual.Mediana, ep_m_anual.baseCalculo).collect()
ipca_mediana_anual_pivot = formata_df(ipca_mediana_anual, '%Y', 'Mediana')
ipca_mediana_anual_pivot

# display(ipca_media_anual_pivot)
# display(ipca_mediana_anual_pivot)

DataReferencia,2024,2025,2026,2027,2028
Data,,,,,
2024-02-29,3.8000,3.5170,3.5,3.5,3.50
2024-03-01,3.7625,3.5145,3.5,3.5,3.50
2024-03-04,3.7619,3.5046,3.5,3.5,3.50
2024-03-05,3.7617,3.5044,3.5,3.5,3.50
2024-03-06,3.7622,3.5044,3.5,3.5,3.50
2024-03-07,3.7622,3.5042,3.5,3.5,3.50
2024-03-08,3.7718,3.5050,3.5,3.5,3.50
2024-03-11,3.7718,3.5042,3.5,3.5,3.50
2024-03-12,3.8300,3.5042,3.5,3.5,3.49


In [153]:
ipca_ult5dias = ep_m_anual.query().filter(ep_m_anual.Indicador == 'IPCA', ep_m_anual.Data >= '2023-01-01', ep_m_anual.baseCalculo == '1').select(ep_m_anual.Indicador, ep_m_anual.Data, ep_m_anual.DataReferencia, ep_m_anual.Mediana. ep_m_anual.baseCalculo).collect()
ipca_ult5dias

AttributeError: 'ODataProperty' object has no attribute 'ep_m_anual'

In [134]:
# Expec inflacao IPCA

# 12 meses -> mediana
ipca_12meses = ep_12meses.query().filter(ep_12meses.Indicador == 'IPCA', ep_m_anual.Data >= '2023-01-01').select(ep_12meses.Indicador, ep_12meses.Data, ep_12meses.Mediana, ep_24meses.Suavizada).collect()
ipca_12meses_pivot = ipca_12meses.pivot_table(index='Data', columns='Suavizada', values='Mediana')
ipca_12meses_pivot.insert(loc=0, column='12 Meses', value='')

# 13 a 24 meses -> mediana
ipca_24meses = ep_24meses.query().filter(ep_24meses.Indicador == 'IPCA', ep_m_anual.Data >= '2023-01-01').select(ep_24meses.Indicador, ep_24meses.Data, ep_24meses.Mediana, ep_24meses.Suavizada).collect()
ipca_24meses_pivot = ipca_24meses.pivot_table(index='Data', columns='Suavizada', values='Mediana')
ipca_24meses_pivot.insert(loc=0, column='13-24 Meses', value='')

ipca_concat = pd.concat([ipca_12meses_pivot, ipca_24meses_pivot], axis=1)
ipca_concat.rename(columns={'N': 'Não Suav', 'S': 'Suav'})
ipca_concat.columns.name=''
ipca_concat


,12 Meses,N,S,13-24 Meses,N,S
Data,,,,,,
2023-01-02,,5.34595,5.40450,,3.89080,3.86335
2023-01-03,,5.34265,5.39090,,3.88180,3.83140
2023-01-04,,5.33105,5.34800,,3.80930,3.79610
2023-01-05,,5.35075,5.39790,,3.88420,3.86730
2023-01-06,,5.36125,5.41795,,3.87630,3.82135
...,...,...,...,...,...,...
2024-03-18,,3.40010,3.42790,,3.59300,3.60100
2024-03-19,,3.40195,3.43605,,3.59665,3.60295
2024-03-20,,3.38890,3.43640,,3.59325,3.59950


In [147]:
# Passando df para planhilha excel

with pd.ExcelWriter(r'C:\Users\rm170600\Documents\Python\Projeto_API_SGS\Expec_Inflacao.xlsx') as writer:
   ipca_mediana_anual_pivot.to_excel(writer, sheet_name='IPCA Mediana Anual')
   ipca_media_anual_pivot.to_excel(writer, sheet_name='IPCA Media Anual')
   ipca_top5_anual_pivot.to_excel(writer, sheet_name='IPCA Top5 Anual')
   df_ipca_ult5dias.to_excel(writer, sheet_name='IPCA ult 5 dias')
   ipca_concat.to_excel(writer, sheet_name='Excp Inflação Suav')